In [18]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input, EfficientNetB7
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from PIL import Image
import numpy as np
import os
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [2]:
def crop_center_square(img):
    # 이미지의 크기를 가져옵니다.
    min_dim = min(img.size)
    # 중앙에서 정사각형으로 자르기 위한 시작점을 계산합니다.
    start_x = (img.size[0] - min_dim) // 2
    start_y = (img.size[1] - min_dim) // 2
    # 이미지를 자릅니다.
    return img.crop((start_x, start_y, start_x + min_dim, start_y + min_dim))

In [3]:
def build_feature_extractor():
    model = EfficientNetB7(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(600, 600, 3),
    )
    model.trainable = False
    
    image_input = keras.Input((600, 600, 3))
    image_features = model(image_input)
    output = Dense(10, activation="softmax")(image_features)

    return Model(inputs=image_input, outputs=output, name="feature_extractor_efficientnet")

In [4]:
train_dir = 'data/train'
# 비디오 파일 목록과 태그를 포함하는 리스트를 만드는 함수
def create_data_list(data_dir):
    data_list = []
    for item in os.listdir(data_dir):
        item_path = os.path.join(data_dir, item)
        if os.path.isdir(item_path):
            for file_name in os.listdir(item_path):
                if file_name.endswith('.jpg'):
                    data_list.append((item, str(data_dir+'/'+item)+'/'+file_name))
    return data_list

train_list = create_data_list(train_dir)
train_df = pd.DataFrame(data=train_list, columns=['tag', 'image_name'])
train_df

,tag,image_name
0,belly,data/train/belly/20231206_121209_001.jpg
1,belly,data/train/belly/20231206_121209_002.jpg
2,belly,data/train/belly/20231206_121209_003.jpg
3,belly,data/train/belly/20231206_121209_004.jpg
4,belly,data/train/belly/20231206_121209_005.jpg
...,...,...
1662,where,data/train/where/20231206_115431_012.jpg
1663,where,data/train/where/20231206_115431_013.jpg
1664,where,data/train/where/20231206_115431_014.jpg
1665,where,data/train/where/20231206_115431_015.jpg


In [5]:
# CSV 파일로 저장
csv_filename = 'image_data.csv'
train_df.to_csv(csv_filename, index=False)

# CSV 파일 불러오기
df = pd.read_csv('image_data.csv')

# 이미지 데이터와 레이블을 저장할 리스트
image_data = []
labels = []

In [6]:
for index, row in df.iterrows():
    img = Image.open(row['image_name'])
    img = crop_center_square(img)
    img = img.resize((600, 600))   # EfficientNetB7에 적합한 크기로 조정
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)  # 이미지 전처리
    image_data.append(img_array)
    labels.append(row['tag'])

# 넘파이 배열로 변환
image_data = np.array(image_data)
labels = np.array(labels)

In [30]:
# 이미지 데이터와 레이블을 압축된 npz 파일로 저장
np.savez_compressed('image_data.npz', image_data=image_data, labels=labels)

In [20]:
# 저장된 npz 파일 로드
data = np.load('image_data.npz')

# 데이터를 다시 넘파이 배열로 추출
image_data = data['image_data']
labels = data['labels']

In [21]:
model = build_feature_extractor()

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
# 먼저 데이터를 셔플링
indices = np.arange(len(image_data))
np.random.shuffle(indices)
image_data = image_data[indices]
labels = labels[indices]

In [24]:
# 레이블 인코더 생성
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(labels)

In [25]:
# 정수 라벨을 원-핫 인코딩으로 변환
labels_one_hot = to_categorical(integer_encoded_labels, num_classes=10)

In [29]:
# stratify를 사용하여 데이터를 분할
image_data_train, image_data_val, labels_train, labels_val = train_test_split(
    image_data, labels_one_hot, test_size=0.2, stratify=labels)

In [30]:
datagen = ImageDataGenerator()

In [31]:
batch_size = 16  # 예를 들어, 배치 크기를 16으로 줄임
# 학습 데이터 생성기
train_generator = datagen.flow(x=image_data_train, y=labels_train,batch_size=batch_size)
# 검증 데이터 생성기
validation_generator = datagen.flow(x=image_data_val, y=labels_val, batch_size=batch_size)

# 콜백 정의
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# 모델 학습
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=200,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/200




ResourceExhaustedError: Graph execution error:

Detected at node feature_extractor_efficientnet/efficientnetb7/block2a_expand_activation/mul_1 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\rlarn\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\rlarn\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\rlarn\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\rlarn\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\rlarn\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\rlarn\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\rlarn\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\rlarn\anaconda3\Lib\site-packages\ipyk

In [ ]:
# 예측 수행
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

# 예측된 정수 라벨을 문자열 라벨로 변환
predicted_labels = label_encoder.inverse_transform(predicted_classes)